In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
tf.reset_default_graph()

train_x = np.loadtxt('1000_index_x.txt')
train_y = np.loadtxt('1000_index_y.txt')


In [2]:
nclasses = int(max(train_x[:,0]))
nsteps = 4
dimhidden = 100
dimoutput = nclasses

x = tf.placeholder("int32", [None, nsteps])
y = tf.placeholder("int32", [None])
x_one_hot = tf.one_hot(x, nclasses, on_value=1.0, off_value=0.0 , dtype='float')
y_one_hot = tf.one_hot(y, nclasses, on_value=1.0, off_value=0.0, dtype='float')

cell = tf.contrib.rnn.BasicLSTMCell(dimhidden)
outputs, states = tf.nn.dynamic_rnn(cell, x_one_hot,  dtype=tf.float32)
last_output = outputs[:, -1, :]


print(outputs.shape)

(?, 4, 100)


In [7]:
w = tf.Variable(tf.random_normal([dimhidden, dimoutput]))
b = tf.Variable(tf.random_normal([dimoutput]))
prediction = tf.nn.softmax(tf.matmul(last_output, w) + b)


In [9]:
cost = -tf.reduce_mean( y_one_hot*tf.log(prediction+0.00000001))
optm     = tf.train.AdamOptimizer().minimize(cost) 
accr     = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(prediction,1), tf.argmax(y_one_hot,1)), tf.float32))


sess = tf.Session()
sess.run(tf.global_variables_initializer())

x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)


for i in range(100):
    _, _cost, _accr = sess.run([optm, cost, accr], feed_dict={x: x_train, y: y_train})
    if i%10 == 0 :
        print(i, " : ", _cost , _accr)
    
    if i%50 == 0 :
        _cost, test_acc = sess.run([cost, accr], feed_dict={x: x_test, y: y_test})
        print( "Test accuracy: %.3f" % (test_acc))


0  :  0.000214769 0.0
Test accuracy: 0.000


KeyboardInterrupt: 